# Basic Resource Provisioning with Terraform 

In this lesson, you will dive into the practical aspects of using Terraform to provision and manage cloud resources. You will learn how to create resources such as Azure Storage Accounts, understand Terraform state management and use essential Terraform commands for resource provisioning and removal.

IMPORTANT: Any resources you provision on Azure will accrue a cost. With an Azure Free Tier account you are allowed a certain capacity free for each resource, which you can learn more about here: [Azure Free Tier account services](https://azure.microsoft.com/en-gb/free/search/?ef_id=_k_CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE_k_&OCID=AIDcmmiouhop3i_SEM__k_CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE_k_&gad_source=1&gclid=CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE).
Once you run out of credits on your free tier subscription you will be charged, so make sure to decomission any resources you create on this account once you are finished with them.

## Hands-On: Creating and Managing Azure Resources

Resource provisioning is a core concept in Terraform, central to its purpose of automating infrastructure management. It involves creating and managing infrastructure resources in a consistent and repeatable manner. To create resources in Terraform, you define them using resource blocks within your Terraform configuration.

In this section, we will walk through an end-to-end example of defining an Azure Storage Account resource. We will demonstrate how to declare resources using resource blocks, understand resource arguments and attributes, and leverage the power of Terraform and Azure together.

Before creating the Azure Storage Account resource, make sure you have an Azure account (if you don't have one, you can sign up for a free Azure trial account) and Azure CLI installed on your local machine.

### Create a New Directory for Your Terraform Project

Begin by creating a new directory for your Terraform project. This directory will contain your configuration files. You can name the directory as you like, for example, `my-terraform-project`.

### Create a `main.tf` Configuration File

Create a new file named `main.tf` within your project directory. You can use Visual Studio Code or the command line to create this file. In this file, you will define your Terraform configurations.

### Create a Service Principal

> IMPORTANT: If you are already working on your specialisation project and using AiCore-generated Azure credentials, you do not need to follow the steps above to create a Service Principal. Instead, you will receive all the necessary credentials (client ID, client secret, subscription ID and tenant ID) when you receive your initial Azure login credentials email from AiCore. Additionally, you will not have permissions to create a new Service Principal in this case, thus the instructions below are only for personal Azure accounts.

To authenticate Terraform with your Azure account, you'll need to create a *Service Principal*, a service account that Terraform uses to interact with Azure resources.

Open your terminal and sign in to Azure using the Azure CLI command: `az login`. Follow the prompts to authenticate with your Azure account. Once you are logged in, list your Azure subscriptions and their details using the following command: `az account list --output table`. This command will display a table with information about your Azure subscriptions. Look for the `SubscriptionId` column to find your subscription ID.

Create a Service Principal by running the following command, replacing `{myApp-name}` with a name for your Service Principal (for example `myAppMaya`) and `{your-subscription-id}` with your Azure subscription ID: `az ad sp create-for-rbac --name {myApp-name} --role contributor --scopes /subscriptions/{your-subscription-id}`.

> DO NOT name your application simply `myApp`, make it unique!

Replace `{your-subscription-id}` with the subscription ID identified in the previous steps. Replace `{your-resource-group-name}` with the name of an existing resource group in your subscription. If you don't have any create one first then proceed with running the command above.

If you encounter a `Insufficient privileges to complete the operation` error when running this command with your personal Azure credentials, follow these steps to grant the necessary permissions:

1. Go to the Azure Portal and use the search bar to access the **Subscriptions** service. Select your subscription (you should have only one available).

2. In the **Subscription** home page, navigate to the **Access control** tab and click **Add role assignment**

<p align="center">
    <img src="images/AccessControl.png" height="250" width="800"/>
</p>

3. In the **Role** tab, search for and select **Contributor** under the **Privileged administrator roles**

<p align="center">
    <img src="images/PrivilegedRole.png" height="300" width="800"/>
</p>

4. In the **Members** tab, select the **User, group, or service principal** option and click the **+ Select members** button. Search for your service principal by name, in this example `{myApp-Name}`. Review and create the assignment.

To retrieve the credentials for your Service Principal, follow these steps. You'll need to note these credentials as they will be used in your Terraform configuration:

- To retrieve the `displayName` of the service principal, you can use the following command. Replace `{myApp-Name}` with the actual name of your service principal. Here's the command:

`az ad sp list --display-name {myApp-Name} --query "[0].appDisplayName"`


- List the service principals (SPs) with the specific display name, in this case, `{myApp-Name}`:

`az ad sp list --display-name {myApp-Name}`

- Note down the value under the `id` field 

- Once you have the ID, you can use it in the `az ad sp credential reset` command to reset the credentials for your service principal. Here's the command:

`az ad sp credential reset --id <your-service-principal-id`

After running the command, you will receive `JSON` output with information about the Service Principal. Note down the values for `appId`, `password`, and `tenant`, as they will be used in your Terraform configuration.

Instead, after running the command above, you will receive `JSON` output with information about the Service Principal. Note down the values for `appId`, `password`, `tenant`, and `displayName`, as they will be used in your Terraform configuration.

### Define the Azure Provider 

Now that you have the required authentication details from the Service Principal, you can define the Azure provider in your Terraform configuration. Open the `main.tf` file in a text editor, and add the following block to define the Azure provider:

```hcl
terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "=3.95.0"
    }
  }
}

provider "azurerm" {
  features {}
  skip_provider_registration = true
  client_id       = "your-client-id"
  client_secret   = "your-client-secret"
  subscription_id = "your-subscription-id"
  tenant_id       = "your-tenant-id"
}
```

- The `terraform` block specifies the required provider for the configuration, in this case, the Azure provider from HashiCorp (`azurerm`), and sets the version to `3.0.0`

- The provider `azurerm` block configures the Azure provider. Inside this block:

  - The features `{}` block is optional and is typically used to enable specific provider features. You can add feature configurations here if needed.

  - The `client_id`, `client_secret`, `subscription_id`, and `tenant_id` parameters should be replaced with your actual Azure authentication details:

     - `client_id` represents the Azure Application ID, which identifies your application. Replace this with your previously identified `appID`.

     - `client_secret` is the Azure Application Secret, which serves as the application's password. Replace this with your previously identified `password`.

     - `subscription_id` is your Azure Subscription ID, which links your resources to your subscription. Replace this with your Subscription ID.

     - `tenant_id` is your Azure Tenant ID, representing the Microsoft Entra Directory that manages your Azure resources. Replace this with your previously identified `tenant`.


### Declare an Azure Storage Account

Next, declare an Azure Storage Account resource within the same `main.tf` file. This resource block specifies the type of resource and its configuration settings. You can customize the settings to your requirements. 

```hcl
# Define the Azure resource group resource
resource "azurerm_resource_group" "example" {
  name     = "my-resource-group"
  location = "UK South"
}

# Define the Azure Storage Account resource
resource "azurerm_storage_account" "example" {
  name                     = "<your-unique-storage-account-name"
  resource_group_name      = "my-resource-group"
  location                 = "UK South"
  account_tier             = "Standard"
  account_replication_type = "LRS"
  depends_on = [azurerm_resource_group.example]
}
```

This Terraform configuration defines the creation of an Azure resource group named `my-resource-group` in the UK South region and an Azure Storage Account named `mystorageaccount` within that resource group. The attributes specify essential details such as resource names, locations, performance tiers, and replication types. Additionally, the `depends_on` attribute establishes a dependency relationship, ensuring that the storage account creation occurs only after the resource group is created, guaranteeing the correct placement of the storage account within the resource group.

Save the `main.tf` file after adding the Azure Resource Group and the Azure Storage Account resource definition.

> A common error when working with Terraform configurations is to get a Resource Dependency Error.

Imagine the following scenario, where you're provisioning a Virtual Machine that relies on a virtual network, but Terraform fails due to the resource creating order. You will likely receive a similar error message:

```shell
Error Message: "Error: Error creating Virtual Machine: Error creating Virtual Machine 'example-vm': Network Interface 'example-nic' has not been created yet."
```

The solution here would be to utilize Terraform's dependency management features like `depends_on` to ensure resources are created in the correct order.

### Initialize a New Terraform Project

First, you will need to install Terraform on your local machine. You can do so by following the Terraform [recommended installation steps](https://developer.hashicorp.com/terraform/downloads). You can check if the installation was successful by running `terraform version`.

Next, initialize your Terraform project within `my-terraform-project` directory using the following command: `terraform init`. This command initializes your project, downloading the necessary provider plugins and setting up the working directory. You should see a similar output if the initialization was successful:

<p align="center">
    <img src="images/TerraformInit.png" height="250" width="600"/>
</p>

### Apply the Configuration

Now, you can apply the configuration to create the Azure Storage Account by running the following command: `terraform apply`. Review the execution plan and confirm the changes by typing **yes** when prompted. Terraform will provision the Azure Storage Account based on your configuration. If everything is created successfully, you should see a similar message in your terminal:

<p align="center">
    <img src="images/ResourcesCreated.png" height="250" width="600"/>
</p>

Additionally, you should be able to see these resources on the Azure portal.

Congratulations! You've successfully created a `main.tf` configuration file, defined the Azure provider, and declared an Azure Storage Account resource using Terraform.

## Terraform Commands

We have seen in the hands-on that Terraform provides different commands that allow you to manage your infrastructure as code (IaC) projects efficiently. In this section we will have a closer look at Terraform commands and how to use them effectively.

### `terraform init`

The `terraform init` command is the first step when starting a new Terraform project or when working on an existing one. It initializes the working directory and downloads the necessary provider plugins and modules.

Run `terraform init` whenever you create a new Terraform configuration or if you've updated your configuration to include new providers or resources.

### `terraform plan`

The `terraform plan` command is used to preview the changes Terraform will make to your infrastructure. It analyzes your configuration and the current state, then shows the proposed additions, modifications, and deletions of resources.

Effective usage of `terraform plan` command includes:

- Run `terraform plan` before applying changes to your infrastructure to review and verify the expected modifications
- Use the `-out` flag to save the plan to a file (`terraform plan -out=plan.tfplan`) and apply the plan later with `terraform apply`

### `terraform apply`

The `terraform apply` command is used to apply the changes specified in your Terraform configuration. It creates, updates, or deletes resources to match the desired state.

Effective usage of the `terraform apply` command includes:

- Always review the output of `terraform plan` before applying changes to avoid unintended modifications
- You can use the `-auto-approve` flag (`terraform apply -auto-approve`) to automatically approve and apply changes, but exercise caution when using this option
- It's a good practice to log or capture the output of `terraform apply` for auditing and tracking changes

### `terraform destroy`

The `terraform destroy` command is used to destroy all resources managed by Terraform in your configuration. It prompts for confirmation before destroying each resource.

Effective usage of the `terraform destroy` command includes:

- Be cautious when running `terraform destroy` in production environments. Ensure that you have a backup or a plan for resource recovery.
- Use the `-target` flag to specify specific resources to destroy (`terraform destroy -target=resource_type.resource_name`) instead of destroying the entire infrastructure
- Capture the plan using `terraform plan -destroy` to review and verify the destruction plan before executing it

> Navigate to the directory of the hands-on Terraform project and destroy all the resources we have provisioned in the hands-on. This ensures you won't incur any charges on your Azure account.

### Terraform Workflow

An effective Terraform workflow involves the following steps:

- **Initialization**: Start by running `terraform init` to set up your project with the required providers and modules
- **Configuration**: Define your infrastructure as code using Terraform configurations (`.tf` files)
- **Planning**: Use `terraform plan` to preview the changes Terraform will make to your infrastructure. Review the plan to ensure it aligns with your intentions.
- **Apply Changes**: Execute `terraform apply` to apply the changes specified in your configuration. Confirm the plan if prompted.
- **Maintenance**: Continuously update and maintain your Terraform configurations as infrastructure requirements evolve
- **Destroy**: When necessary, use `terraform destroy` to tear down resources. Be cautious and confirm the destruction plan.

##  Key Takeaways

- Terraform enables you to create and manage infrastructure resources in a consistent and repeatable manner. It automates the process of provisioning resources, making infrastructure management more efficient and error-resistant.
- Resources are defined in Terraform using resource blocks within your configuration. A `resource` block specifies the resource type and a user-defined resource name.
- Resource attributes and arguments allow you to customize resource properties. You can configure properties such as name, location, and dependencies within resource blocks
- Terraform provides a set of essential commands, including `init` for project initialization, `plan` for generating execution plans, `apply` for applying configurations, and `destroy` for resource cleanup